**Dependencies:**

- To direct c++ std stream to Jupyter notebook:
    `pip install wurlitzer`

- Deformed frame visualization
    Install `meshcat-python`: https://github.com/rdeits/meshcat-python

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import pyconmech as cm
from wurlitzer import sys_pipes

import meshcat
import meshcat.geometry as g
from deformed_frame_viz import meshcat_visualize_deformed

In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


# check deformation for a completed structure

In [10]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,"tower_3D.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio=10.0
time_step = 1.0

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
    sc.set_output_json(True)
    sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

    # manually assigned point loads
#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     ext_load[0,4] = -1
#     include_sw = False

    # import load from a json file
    ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
    print('external load:')
    print(ext_load)
    print('include self weight : {0}'.format(include_sw))
    sc.set_load(nodal_forces = ext_load)
    sc.set_self_weight_load(include_sw)
    
    sc.solve()
    
    # visualize deformed structure
    orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
    beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
    meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=0.5)

external load:
[[   5.       0.      11.125    0.       0.       0.       0.   ]
 [   7.       0.      11.125    0.       0.       0.       0.   ]
 [   8.     222.5     22.25  -111.25     0.       0.       0.   ]
 [   9.     222.5      0.    -111.25     0.       0.       0.   ]]
include self weight : False
computed result file saved successfully!


# animate deformation for a construction sequence

In [11]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,"tower_3D.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio=1.0
time_step = 1.0

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
    
#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     ext_load[0,4] = -1
#     include_sw = False

#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw)

    sc.set_self_weight_load(True)

    for i in range(0,24):
        vis.delete()
        existing_ids = list(range(0,i+1))
        sc.solve(existing_ids)

        orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
        beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
        meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=0.5)
        time.sleep(time_step)